In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

sheet = pd.read_excel('data_j.xls')
sheet

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20210730,1301,極洋,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20210730,1305,ダイワ上場投信－トピックス,ETF・ETN,-,-,-,-,-,-
2,20210730,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN,-,-,-,-,-,-
3,20210730,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
4,20210730,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
4128,20210730,9993,ヤマザワ,市場第一部（内国株）,6100,小売業,14,小売,7,TOPIX Small 2
4129,20210730,9994,やまや,市場第一部（内国株）,6100,小売業,14,小売,7,TOPIX Small 2
4130,20210730,9995,グローセル,市場第一部（内国株）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4131,20210730,9996,サトー商会,JASDAQ(スタンダード・内国株）,6050,卸売業,13,商社・卸売,-,-


In [2]:
# テーブルの形式に合わせる

new_s = sheet[['コード', '銘柄名']]
id_num = pd.RangeIndex(start=1, stop=len(new_s.index) + 1, step=1)
new_s['id'] = id_num
new_s = new_s.reindex(columns=['id', 'コード', '銘柄名'])
new_s = new_s.rename(columns={'コード': 'code', '銘柄名': 'name'})
new_s

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,code,name
0,1,1301,極洋
1,2,1305,ダイワ上場投信－トピックス
2,3,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信
3,4,1308,上場インデックスファンドＴＯＰＩＸ
4,5,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信
...,...,...,...
4128,4129,9993,ヤマザワ
4129,4130,9994,やまや
4130,4131,9995,グローセル
4131,4132,9996,サトー商会


In [3]:
# INSERTとSELECT

import sqlite3
import pandas.io.sql as psql
with sqlite3.connect('db.sqlite3') as conn:
    try:
        new_s.to_sql('palst_company', conn, if_exists='replace', index=None)
    except Exception as e:
        print(e)
    
with sqlite3.connect('db.sqlite3') as conn:
    try:
        company = pd.read_sql_query(sql=u"SELECT * FROM palst_company", con=conn)
    except Exception as e:
        print(e)

# sqliteにファイルから直接接続する方法。リレーションは直で設定しないといけないため。
import sys
import os
import django

sys.path.append('api/')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'api.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

company

,id,code,name
0,1,1301,極洋
1,2,1305,ダイワ上場投信－トピックス
2,3,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信
3,4,1308,上場インデックスファンドＴＯＰＩＸ
4,5,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信
...,...,...,...
4128,4129,9993,ヤマザワ
4129,4130,9994,やまや
4130,4131,9995,グローセル
4131,4132,9996,サトー商会


In [4]:
# 会社毎の株価取得
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

def get_stock_data(code=None, S_year=5, S_day=1): 
    if code is None:
        return None

    company_code = str(code) + '.T'
    my_share = share.Share(company_code)
    symbol_data = None
    
    try:
        symbol_data = my_share.get_historical(
            share.PERIOD_TYPE_YEAR,
            S_year,
            share.FREQUENCY_TYPE_DAY,
            S_day)
    except YahooFinanceError as e:
        print(e.message)
        return None
    df_base = pd.DataFrame(symbol_data)
    df_base = pd.DataFrame(symbol_data.values(), index=symbol_data.keys()).T
    df_base.timestamp = pd.to_datetime(df_base.timestamp, unit='ms')
    df_base.index = pd.DatetimeIndex(df_base.timestamp, name='timestamp').tz_localize('UTC').tz_convert('Asia/Tokyo')
    df_base = df_base.reset_index(drop=True)
    return df_base


import time

start = time.time()

cnt = 0

df = None

from palst.models import *
objects = Company.objects.all()
for obj in objects:
    if cnt > 10:
        break
        
    cnt += 1
    
    new_df = get_stock_data(obj.code, 1, 1)
    if new_df is None:
        continue
    
    new_df['id'] = new_df.index
    new_df['code'] = obj.code
    
    if df is not None:
        df = pd.concat([df, new_df])

    if df is None:
        df = new_df
        
#     print("id:[{}], code:[{}], name:[{}]".format(obj.id, obj.code, obj.name))
    
    
end = time.time()

print('処理時間 : {}'.format(end-start))

処理時間 : 2.597581148147583


In [6]:
df = df.dropna()
df

,timestamp,open,high,low,close,volume,id,code
0,2020-08-06 00:00:00,2712.0,2746.0,2706.0,2744.0,9200.0,0,1301
1,2020-08-07 00:00:00,2750.0,2763.0,2675.0,2675.0,31100.0,1,1301
2,2020-08-11 00:00:00,2661.0,2690.0,2607.0,2672.0,28100.0,2,1301
3,2020-08-12 00:00:00,2672.0,2716.0,2654.0,2701.0,19700.0,3,1301
4,2020-08-13 00:00:00,2712.0,2712.0,2680.0,2712.0,16600.0,4,1301
...,...,...,...,...,...,...,...,...
237,2021-07-28 00:00:00,28360.0,28510.0,28150.0,28280.0,263250.0,237,1321
238,2021-07-29 00:00:00,28460.0,28520.0,28350.0,28490.0,159982.0,238,1321
239,2021-07-30 00:00:00,28330.0,28380.0,27960.0,27990.0,395786.0,239,1321
240,2021-08-02 00:00:00,28240.0,28530.0,28210.0,28470.0,238943.0,240,1321


In [14]:
with sqlite3.connect('db.sqlite3') as conn:
    try:
        df.to_sql('palst_stock', conn, if_exists='replace', index=None)
    except Exception as e:
        print('登録失敗 id:{} code:{}'.format(obj.id, obj.code))

In [15]:
from palst.models import *
objects = Stock.objects.all()
# for obj in objects:
#     print("id:[{}], code:[{}], close:[{}]".format(obj.id, obj.code, obj.close))

Stock.objects.all().count()

2665